# Gaussova eliminacija
---

## Općenito

Sustav $Ax=b$
se rješava u tri koraka (__bez pivotiranja__):

1. $A=LU$ (LU rastav, $O\big(\displaystyle\frac{2}{3}n^3\big)$ operacija),
2. $Ly=b$ (donje trokutrasti sustav, $n^2$ operacija),
3. $Ux=y$ (gornje torkutasti sustav, $n^2$ operacija).

Ukoliko pivotiramo, tada je

1. $PA=LU$, 
2. $Ly=P^T b$,
3. $Ux=y$. 

## LU rastav

In [1]:
function mylu{T}(A1::Array{T}) # Strang, page 100
    A=deepcopy(A1)
    n,m=size(A)
    U=map(Float64,[zero(A[1,1]) for i=1:n, j=1:n]) # This acccepts blocks and numbers
    L=map(Float64,[zero(A[1,1]) for i=1:n, j=1:n])
    for k=1:n
        L[k,k]=one(A[1,1])
        for i=k+1:n
            L[i,k]=A[i,k]/A[k,k]
            for j=k+1:n
                A[i,j]=A[i,j]-L[i,k]*A[k,j]
            end
        end
        for j=k:n
            U[k,j]=A[k,j]
        end
    end
    L,U
end

mylu (generic function with 1 method)

In [2]:
A=rand(6,6)
# A=[2.0 2;3 4]

6x6 Array{Float64,2}:
 0.653018    0.386455   0.601849   0.0801381  0.73574   0.129083
 0.509359    0.0778841  0.889809   0.759393   0.477421  0.47311 
 0.641238    0.169071   0.858514   0.664296   0.979717  0.378952
 0.508776    0.0946753  0.975459   0.87416    0.844835  0.992135
 0.00376703  0.770302   0.0306788  0.519082   0.250509  0.83525 
 0.775469    0.935316   0.76713    0.987087   0.309463  0.466184

In [3]:
L,U=mylu(A)

(
6x6 Array{Float64,2}:
 1.0          0.0         0.0        0.0     0.0      0.0
 0.780008     1.0         0.0        0.0     0.0      0.0
 0.981961     0.941216    1.0        0.0     0.0      0.0
 0.779115     0.923345   -0.924138   1.0     0.0      0.0
 0.00576865  -3.43573   -11.4841    20.3863  1.0      0.0
 1.18752     -2.131      -7.4004    17.9772  1.04623  1.0,

6x6 Array{Float64,2}:
 0.653018   0.386455   0.601849   0.0801381   0.73574    0.129083 
 0.0       -0.223554   0.420362   0.696885   -0.096462   0.372423 
 0.0        0.0       -0.128131  -0.0703156   0.348041  -0.0983333
 0.0        0.0        0.0        0.103276    0.682314   0.456816 
 0.0        0.0        0.0        0.0        -9.99809   -8.32799  
 0.0        0.0        0.0        0.0         0.0        0.879518 )

In [4]:
L*U-A

6x6 Array{Float64,2}:
 0.0          0.0  0.0  0.0           0.0           0.0        
 0.0          0.0  0.0  0.0           0.0           0.0        
 0.0          0.0  0.0  0.0           0.0           0.0        
 0.0          0.0  0.0  0.0           0.0           0.0        
 4.33681e-19  0.0  0.0  0.0           6.66134e-16   0.0        
 1.11022e-16  0.0  0.0  1.11022e-16  -1.33227e-15  -1.11022e-15

## Trokutasti sustavi

In [6]:
function myU{T}(U::Array{T},b1::Array{T})
    b=deepcopy(b1)
    n=length(b)
    for i=n:-1:1
       for j=n:-1:i+1
            b[i]=b[i]-U[i,j]*b[j]
       end
        b[i]=b[i]/U[i,i]
    end
    b
end

function myL{T}(L::Array{T},b1::Array{T})
    b=deepcopy(b1)
    n=length(b)
    for i=1:n
        for j=1:i-1
            b[i]=b[i]-L[i,j]*b[j]
        end
        b[i]=b[i]/L[i,i]
    end
    b
end

myL (generic function with 1 method)

In [7]:
b=rand(6)

6-element Array{Float64,1}:
 0.518376
 0.298284
 0.124805
 0.291274
 0.595325
 0.461642

In [8]:
# Rijesimo sustav pomocu ugradjene funkcije:
x=A\b

6-element Array{Float64,1}:
 -2.79803  
  1.81014  
  3.00531  
 -1.24616  
 -0.0383612
 -0.268443 

In [9]:
# Rijesimo sustav pomocu nasih funkcija
y=myL(L,b)

6-element Array{Float64,1}:
  0.518376
 -0.106053
 -0.284402
 -0.277503
  2.61913 
 -0.236101

In [10]:
x1=myU(U,y)

6-element Array{Float64,1}:
 -2.79803  
  1.81014  
  3.00531  
 -1.24616  
 -0.0383612
 -0.268443 

In [11]:
# Usporedimo rjesenja
x-x1

6-element Array{Float64,1}:
  4.44089e-16
 -2.22045e-16
  4.44089e-16
 -2.22045e-16
 -1.31839e-16
 -2.22045e-16

## Brzina

Program `mylu()` je jako spor. Između ostalog, alocira nepotrebno tri matrice i ne računa s blok matricama.

Program se može reformulirati na načun da su i $L$ i $U$ spremljene u polje $A$, pri čemu se dijagonala od $L$ ne sprema, jer su svi elementi jednaki 1 (vidi [Introduction to Linear Algebra, str. 100][St09]):

[St09]: https://books.google.hr/books?id=M19gPgAACAAJ&dq=strang%20introduction&hl=hr&source=gbs_book_other_versions "Gilbert Strang, 'Introduction to Linear Algebra, 4th Edition', Wellesley-Cambridge Press, 2009"


In [12]:
function mylu1{T}(A1::Array{T}) # Strang, page 100
    A=deepcopy(A1)
    n,m=size(A)
    for k=1:n-1
        rho=k+1:n
        A[rho,k]=A[rho,k]/A[k,k]
        A[rho,rho]=A[rho,rho]-A[rho,k]*A[k,rho]
    end
    A
end

mylu1 (generic function with 1 method)

In [13]:
mylu1(A)

6x6 Array{Float64,2}:
 0.653018     0.386455    0.601849   0.0801381   0.73574    0.129083 
 0.780008    -0.223554    0.420362   0.696885   -0.096462   0.372423 
 0.981961     0.941216   -0.128131  -0.0703156   0.348041  -0.0983333
 0.779115     0.923345   -0.924138   0.103276    0.682314   0.456816 
 0.00576865  -3.43573   -11.4841    20.3863     -9.99809   -8.32799  
 1.18752     -2.131      -7.4004    17.9772      1.04623    0.879518 

In [14]:
L,U

(
6x6 Array{Float64,2}:
 1.0          0.0         0.0        0.0     0.0      0.0
 0.780008     1.0         0.0        0.0     0.0      0.0
 0.981961     0.941216    1.0        0.0     0.0      0.0
 0.779115     0.923345   -0.924138   1.0     0.0      0.0
 0.00576865  -3.43573   -11.4841    20.3863  1.0      0.0
 1.18752     -2.131      -7.4004    17.9772  1.04623  1.0,

6x6 Array{Float64,2}:
 0.653018   0.386455   0.601849   0.0801381   0.73574    0.129083 
 0.0       -0.223554   0.420362   0.696885   -0.096462   0.372423 
 0.0        0.0       -0.128131  -0.0703156   0.348041  -0.0983333
 0.0        0.0        0.0        0.103276    0.682314   0.456816 
 0.0        0.0        0.0        0.0        -9.99809   -8.32799  
 0.0        0.0        0.0        0.0         0.0        0.879518 )

Usporedimo brzine LAPACK-ovog programa `lu()` i našeg naivnog programa `mylu()`na većoj dimenziji. 

Izvedite program par puta radi točnijeg mjerenja brzine.

In [15]:
n=512
A=rand(n,n)

512x512 Array{Float64,2}:
 0.192478   0.809943   0.855846   …  0.610998   0.526602    0.224027 
 0.0935146  0.852439   0.0613699     0.506184   0.847527    0.120355 
 0.0211303  0.325117   0.633091      0.618872   0.920787    0.131382 
 0.457873   0.613817   0.590023      0.0640139  0.667477    0.777776 
 0.483842   0.12264    0.949276      0.481269   0.331783    0.958864 
 0.957335   0.164872   0.201933   …  0.213216   0.116016    0.86668  
 0.0681968  0.478824   0.237192      0.730866   0.92843     0.748659 
 0.44437    0.483323   0.0234203     0.566899   0.465529    0.0222732
 0.376585   0.183728   0.815143      0.576108   0.0435919   0.901596 
 0.818301   0.677237   0.240211      0.96246    0.0989068   0.396855 
 0.308748   0.0852666  0.608998   …  0.763359   0.776179    0.247996 
 0.850243   0.216705   0.142205      0.574962   0.278542    0.192442 
 0.509391   0.0971858  0.723013      0.576057   0.110299    0.310514 
 ⋮                                ⋱             ⋮               

In [18]:
@time lu(A);

  0.242977 seconds (38 allocations: 6.009 MB)


In [22]:
@time mylu1(A);

  0.404241 seconds (17.26 k allocations: 1.004 GB, 11.09% gc time)


### Blok varijanta

`mylu()` je nekoliko desetaka puta sporiji od `lu()`. `mylu1()` je malo sporiji od `lu()`.

Probajmo s blokovima:

Preradimo `mylu1()` za rad s blokovima (nemamo ugrađeno pivotiranje!)

In [24]:
function mylu2{T}(A1::Array{T}) # Strang, page 100
    A=deepcopy(A1)
    n,m=size(A)
    for k=1:n-1
        for rho=k+1:n
            A[rho,k]=A[rho,k]/A[k,k]
            for l=k+1:n
                A[rho,l]=A[rho,l]-A[rho,k]*A[k,l]
            end
        end
    end
    A
end

mylu2 (generic function with 1 method)

Napravimo prvo mali test:

In [26]:
# Probajte k,l=32,16 i k,l=64,8
k,l=4,4
Ab=[rand(k,k) for i=1:l, j=1:l]

4x4 Array{Any,2}:
 4x4 Array{Float64,2}:
 0.996814   0.51028   0.713424  0.580486 
 0.0595706  0.969683  0.106152  0.0841541
 0.835064   0.181397  0.595753  0.352613 
 0.755699   0.742151  0.143258  0.579418   …  4x4 Array{Float64,2}:
 0.0441784  0.801592    0.576787  0.953355
 0.120594   0.0268804   0.669226  0.852856
 0.257314   0.00726953  0.329245  0.600861
 0.637395   0.363352    0.537672  0.956655
 4x4 Array{Float64,2}:
 0.932054   0.564471  0.79351   0.707334
 0.12528    0.878006  0.747165  0.80843 
 0.362067   0.717824  0.255609  0.697612
 0.0292027  0.636593  0.114183  0.182938         4x4 Array{Float64,2}:
 0.342204  0.368878  0.766023  0.585654
 0.616303  0.830738  0.157813  0.112795
 0.4515    0.389463  0.759445  0.168638
 0.330274  0.830202  0.048666  0.322114            
 4x4 Array{Float64,2}:
 0.726116  0.804829  0.166022  0.191416
 0.842827  0.316111  0.420858  0.512057
 0.412365  0.511836  0.963591  0.287114
 0.498671  0.129929  0.244025  0.795893             4x4 Array

In [27]:
A0=mylu2(Ab)

4x4 Array{Any,2}:
 4x4 Array{Float64,2}:
 0.996814   0.51028   0.713424  0.580486 
 0.0595706  0.969683  0.106152  0.0841541
 0.835064   0.181397  0.595753  0.352613 
 0.755699   0.742151  0.143258  0.579418               …  4x4 Array{Float64,2}:
 0.0441784  0.801592    0.576787  0.953355
 0.120594   0.0268804   0.669226  0.852856
 0.257314   0.00726953  0.329245  0.600861
 0.637395   0.363352    0.537672  0.956655    
 4x4 Array{Float64,2}:
 2.60695  -0.214092  -1.66901  -0.344203 
 7.65175  -0.440027  -7.42453  -1.68841  
 3.63641  -0.423958  -3.84033  -0.0404668
 1.19478   0.392133  -1.26909  -0.165892                  4x4 Array{Float64,2}:
 0.901704  -1.57788     0.140221  -0.384984
 3.31794   -4.62356    -0.608846  -0.730428
 1.35594   -2.47144     0.231901  -0.590359
 0.662495  -0.0685667  -0.395859  -0.230125
 4x4 Array{Float64,2}:
 -2.71237   0.894472   3.12274    1.01743 
  0.37765  -0.231118   0.194184   0.420791
  3.15934   0.439257  -1.85893   -1.60216 
  3.96489  -1.25693 

In [28]:
# Provjera
U=triu(A0)
L=tril(A0)
for i=1:maximum(size(L))
    L[i,i]=eye(L[1,1])
end

In [29]:
Res=L*U-Ab

4x4 Array{Any,2}:
 4x4 Array{Float64,2}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0                                                                                                                                                  …  4x4 Array{Float64,2}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0                                                                                                                                                
 4x4 Array{Float64,2}:
 -1.11022e-16  -5.55112e-16  -6.66134e-16  -4.44089e-16
 -4.44089e-16  -4.44089e-16  -2.22045e-16  -9.99201e-16
  2.22045e-16   1.11022e-16   5.55112e-17   1.11022e-16
  8.32667e-17   0.0           1.94289e-16   0.0             4x4 Array{Float64,2}:
 0.0  0.0          0.0  0.0
 0.0  4.44089e-16  0.0  0.0
 0.0  2.22045e-16  0.0  0.0
 0.0  0.0          0.0  0.0                                                                                                     

In [30]:
# pretvaranje blok matrice u obicnu
unblock(A) = mapreduce(identity, hcat, [mapreduce(identity, vcat, A[:,i]) for i = 1:size(A,2)])

unblock (generic function with 1 method)

In [31]:
norm(unblock(Res))

7.514228677962759e-15

Sada probajmo veću dimenziju:

In [34]:
# Probajmo vece dimenzije (n=k*l)
k,l=32,16
Ab=[rand(k,k) for i=1:l, j=1:l];

In [37]:
@time mylu2(Ab);

  0.281164 seconds (8.79 k allocations: 26.526 MB, 8.39% gc time)


Vidimo da je `mylu2()` gotovo jednako brz kao `lu()`, uz napomenu da `mylu2()` nema ugrađeno pivotiranje. 

## Pivotiranje

Stanardne implementacije uvijek računaju Gaussovu eliminaciju s _parcijalnim pivotiranjem_:

> u svakom koraku se retci pivotiranju tako da pivotni element ima najveću apsolutnu vrijednst u danom stupcu. Na taj 
> način je 
> 
> $$|L_{ij}| \leq 1,$$
> 
> što u praksi dovoljno spriječava rast elemenata.


In [36]:
A=rand(5,5)
L,U,P=lu(A)

(
5x5 Array{Float64,2}:
 1.0        0.0        0.0       0.0       0.0
 0.874008   1.0        0.0       0.0       0.0
 0.563144   0.839234   1.0       0.0       0.0
 0.938453  -0.52993   -0.860008  1.0       0.0
 0.07838   -0.450629  -0.172189  0.858843  1.0,

5x5 Array{Float64,2}:
 0.888449   0.729541   0.827686   0.410832     0.774184 
 0.0       -0.477083  -0.171609   0.493423    -0.385795 
 0.0        0.0        0.411301  -0.00647101   0.860461 
 0.0        0.0        0.0        0.54949      0.805759 
 0.0        0.0        0.0        0.0         -0.0697291,

[1,3,2,5,4])

In [37]:
L*U-A[P,:]

5x5 Array{Float64,2}:
 0.0   0.0           0.0          0.0          0.0
 0.0   0.0           0.0          0.0          0.0
 0.0   0.0           0.0          0.0          0.0
 0.0   0.0           0.0          1.11022e-16  0.0
 0.0  -5.55112e-17  -1.38778e-17  0.0          0.0

### Potpuno pivotiranje

Sljedeći program računa Gaussovu eliminaciju s _potpunim pivotiranjem_ - u svakom koraku retci i stupci zamijene na način da se na pivotnu poziciju dovede element koji ima najveću apsolutnu vrijednost u trenutnoj podmatrici.

In [38]:
function gecp{T}(A1::Array{T})
    # Gaussova eliminacija s potpunim pivotiranjem
    # Izlaz: Pr*L*U*Pc'=A ili Pr'*A*Pc=L*U
    A=deepcopy(A1)
    n,m=size(A)
    Pr=eye(n,n)
    Pc=eye(n,n)
    D=zeros(n)
    for i=1:n-1
        am1,im1=findmax(abs(A[i:n,i:n]),1)
        am,JJ=findmax(am1)
        II=mod(im1[JJ],(n-i+1))
        if II==0
            II=n-i+1
        end
        imax=II+i-1
        jmax=JJ+i-1
        #  zamijena redaka
        if (imax != i)
            temp = Pr[:,i]
            Pr[:,i] = Pr[:,imax]
            Pr[:,imax] = temp
            temp = A[i,:]
            A[i,:] = A[imax,:]
            A[imax,:] = temp
        end
        # zamijena stupaca
        if (jmax != i)
            temp = Pc[:,i]
            Pc[:,i] = Pc[:,jmax]
            Pc[:,jmax] = temp
            temp = A[:,i]
            A[:,i] = A[:,jmax]
            A[:,jmax] = temp
        end
        # eliminacija
        D[i]=A[i,i]
        A[i+1:n,i] = A[i+1:n,i]/D[i]
        A[i+1:n,i+1:n] = A[i+1:n,i+1:n] - A[i+1:n,i]*A[i,i+1:n]
        A[i,i+1:n]=A[i,i+1:n]/D[i]
    end
    D[n]=A[n,n]
    L=eye(n,n)+tril(A,-1)
    U=eye(n,n)+triu(A,1)
    U=diagm(D)*U
    L,U,Pr,Pc
end

gecp (generic function with 1 method)

In [39]:
n=5
A=rand(n,n)
b=rand(n)

5-element Array{Float64,1}:
 0.196101 
 0.0962655
 0.839942 
 0.378201 
 0.443998 

In [40]:
L,U,Pr,Pc=gecp(A)

(
5x5 Array{Float64,2}:
 1.0       0.0       0.0        0.0        0.0
 0.791796  1.0       0.0        0.0        0.0
 0.71467   0.764441  1.0        0.0        0.0
 0.676437  0.264556  0.591514   1.0        0.0
 0.945698  0.823232  0.529724  -0.0315266  1.0,

5x5 Array{Float64,2}:
 0.929529  0.027989   0.565329   0.341097   0.831957 
 0.0       0.691343  -0.259979   0.286085  -0.0858094
 0.0       0.0        0.58509   -0.406798   0.182008 
 0.0       0.0        0.0        0.599241  -0.0524726
 0.0       0.0        0.0        0.0       -0.475278 ,

5x5 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0,

5x5 Array{Float64,2}:
 0.0  0.0  1.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0)

In [41]:
Pr*L*U*Pc'-A

5x5 Array{Float64,2}:
 0.0  0.0   0.0          0.0  0.0
 0.0  0.0   0.0          0.0  0.0
 0.0  0.0   0.0          0.0  0.0
 0.0  0.0   0.0          0.0  0.0
 0.0  0.0  -1.11022e-16  0.0  0.0

In [42]:
y=myL(L,Pr'*b)

5-element Array{Float64,1}:
  0.196101 
 -0.0590062
  0.283161 
  0.555409 
  0.174635 

In [43]:
z=myU(U,y)

5-element Array{Float64,1}:
 -0.52938  
 -0.0422872
  1.22031  
  0.894681 
 -0.367437 

In [44]:
x=Pc*z

5-element Array{Float64,1}:
  1.22031  
 -0.0422872
  0.894681 
 -0.52938  
 -0.367437 

In [45]:
A*x-b

5-element Array{Float64,1}:
  5.55112e-17
  1.11022e-16
  3.33067e-16
  5.55112e-17
 -5.55112e-17

## Točnost

Neka je zadan sustav $Ax=b$, pri čemu je matrica $A$ regularna.

Da bi primijenili koncepte iz bilježnice [NA04_Pogreska_unatrag_i stabilni_algoritmi](NA04_Pogreska_unatrag_i_stabilni_algoritmi.ipynb), potrebno je:

1. napraviti teoriju smetnje za dani problem
2. analizirati pogreške algoritma (Gaussove eliminacije)

### Teorija smetnje

Neka je 

$$
(A+\delta A)\hat x=(b+\delta b)
$$

za neki $\hat x=x+\delta x$.

Želimo ocijeniti 

$$
\frac{\| \hat x - x \|}{\| x\|} \equiv \frac{\| \delta x\|}{\| x\|}.
$$

Uvedimo oznake (npr. prema [Matrix Computations, poglavlje 2.6.2][GVL13])

$$
\delta A=\varepsilon F, \quad \delta b=\varepsilon f, \qquad \hat x=x(\varepsilon),
$$
čime smo dobili jednodimenzionalni problem 

$$
(A+\varepsilon F)x(\varepsilon)=b+\varepsilon f.
$$

za neke (nepoznate) matricu $F$ i vektor $f$. 

Deriviranje po $\varepsilon$ daje

$$
Fx(\varepsilon)+(A+\varepsilon F)\dot x(\varepsilon)=f.
$$

Uvrštavanje $\varepsilon=0$ daje

$$
F x+A\dot x(0)=f,
$$

odnosno

$$
\dot x(0)=A^{-1}(f-Fx).
$$

Taylorov razvoj oko $x(0)$ glasi

$$
x(\varepsilon)=x(0)+\varepsilon \dot x(0) +O(\varepsilon^2),
$$

odnosno, uz zanemarivanje člana $O(\varepsilon^2)$,

$$
\hat x-x=\varepsilon A^{-1}(f-Fx)=A^{-1} (\varepsilon f + \varepsilon F x) = A^{-1} (\delta b + \delta A x).
$$

Svojstva norme povlače

$$
\| \hat x-x\|\leq \| A^{-1} \| (\| \delta b \|  + \| \delta A \| \cdot \|  x\| ).
$$

Konačno, zbog $\| b\| \leq \| A\| \| x\|$, imamo

$$
\frac{\| \hat x-x\|}{\| x\|}\leq \| A\|  \cdot \| A^{-1} \| \bigg(\frac{\| \delta b \|}{\|b\|}  + \frac{\| \delta A \|}{ \|  A\|} \bigg). \tag{1}
$$

Broj 
$$
\kappa(A)\equiv \| A\|  \cdot \| A^{-1} \|
$$ 

je __uvjetovanost__ (__kondicija__)  matrice $A$ i kazuje nam 

> koliko se relativno uvećaju relativne promjene u polaznim podacima (matrici $A$ i vektoru $b$).

Pogledajmo primjer iz [Numeričke matematike, str. 42][RS04]:


[GVL13]: https://books.google.hr/books?id=X5YfsuCWpxMC&printsec=frontcover&hl=hr#v=onepage&q&f=false "G. Golub and C. F Van Loan, 'Matrix Computations', 4th Edition, John Hopkins, Baltimore, 2013" 

[RS04]: http://www.mathos.unios.hr/pim/Materijali/Num.pdf "R. Scitovski, 'Numerička matematika', Sveučilište u Osijeku, Osijek, 2004."

In [46]:
A= [0.234 0.458; 0.383 0.750]

2x2 Array{Float64,2}:
 0.234  0.458
 0.383  0.75 

In [47]:
b=[0.224;0.367]

2-element Array{Float64,1}:
 0.224
 0.367

In [48]:
x=A\b

2-element Array{Float64,1}:
 -1.0
  1.0

In [49]:
δb=[0.00009; 0.000005]
x1=A\(b+δb)

2-element Array{Float64,1}:
 -0.241744
  0.612791

In [50]:
cond(A), norm(δb)/norm(b), norm(x1-x)/norm(x)

(11322.197586092605,0.0002096449170953002,0.6020311134825742)

In [51]:
δA=[-0.001 0;0 0]
x2=(A+δA)\b

2-element Array{Float64,1}:
 0.129518
 0.423193

In [52]:
cond(A), norm(δA)/norm(A), norm(x2-x)/norm(x)

(11322.197586092605,0.0010134105230118603,0.896804787832142)

### Pogreška Gaussove eliminacije

Prema [Matrix Computations, poglavlje 3.3][GVL13], za izračunate faktore
$\hat L$ i $\hat U$ vrijedi

$$
\hat L\cdot \hat U = A+\delta A
$$

gdje je (nejednakost se čita po elementima matrica, $\varepsilon$ je sada točnost stroja)

$$
| \delta A|\leq 3(n-1) \varepsilon (|A|+|\hat L| \cdot |\hat U|) +O(\varepsilon^2).
$$

Zanemarivanje člana $O(\varepsilon^2)$ i prelazak na normu daju

$$
\|\delta A \| \approx \leq  O(n)\varepsilon (\| A\| + \| \hat L\| \cdot \| \hat U\|),
$$

pa je 

$$
 \frac{\|\delta A \|}{\|A\|} \leq O(n)\varepsilon \bigg(1+\frac{\| \hat L\| \cdot \| \hat U\|}{\|A\|}\bigg).
$$

Ukoliko se Gaussova eliminacija radi s pivotiranjem, tada će najvjerojatnije zadnji kvocijent također biti malen 
($\approx 1$). Također, pogreška kod rješavanja trokutastih sustava nije veća od navedene pa, uvrštavanjme u (1), slijedi 
da za relativnu pogrešku izračunatog rješenja vrijedi

$$
\frac{\| \hat x-x\|}{\| x\|}\leq \kappa(A) O(n\varepsilon).
$$

> __Ukoliko je kondicija matrice velika, rješenje može biti netočno.__

[GVL13]: https://books.google.hr/books?id=X5YfsuCWpxMC&printsec=frontcover&hl=hr#v=onepage&q&f=false "G. Golub and C. F Van Loan, 'Matrix Computations', 4th Edition, John Hopkins, Baltimore, 2013" 

In [53]:
n=10
v=rand(n)

10-element Array{Float64,1}:
 0.607348
 0.240677
 0.703843
 0.281695
 0.649881
 0.184879
 0.42333 
 0.636226
 0.517925
 0.878924

In [54]:
# Vandermonmde-oove matrice imaju veliku kondiciju.
A=Array(Float64,n,n)
for i=1:n
    A[:,i]=v.^(i-1)
end
A=A'

10x10 Array{Float64,2}:
 1.0        1.0          1.0        …  1.0        1.0         1.0     
 0.607348   0.240677     0.703843      0.636226   0.517925    0.878924
 0.368871   0.0579256    0.495395      0.404783   0.268247    0.772507
 0.224033   0.0139414    0.34868       0.257533   0.138932    0.678975
 0.136066   0.00335537   0.245416      0.163849   0.0719563   0.596768
 0.0826393  0.000807562  0.172734   …  0.104245   0.037268    0.524513
 0.0501908  0.000194362  0.121578      0.0663235  0.0193021   0.461007
 0.0304833  4.67785e-5   0.0855717     0.0421967  0.00999703  0.40519 
 0.0185139  1.12585e-5   0.0602291     0.0268466  0.00517772  0.356131
 0.0112444  2.70967e-6   0.0423918     0.0170805  0.00268167  0.313012

In [55]:
b=rand(n)

10-element Array{Float64,1}:
 0.176222 
 0.514327 
 0.170147 
 0.452078 
 0.0455566
 0.852881 
 0.720554 
 0.602203 
 0.712276 
 0.798477 

In [56]:
x=A\b

10-element Array{Float64,1}:
     7.0861e7 
     1.58042e6
    -5.17549e6
    -2.1107e6 
     1.19591e8
    -2.57728e5
     3.65903e6
    -1.77173e8
    -1.09871e7
 12197.8      

In [57]:
cond(A)

7.492325039096141e9

In [58]:
Ab=Array(BigFloat,n,n)
bb=Array(BigFloat,n)
for i=1:n
    for j=1:n
        Ab[i,j]=convert(BigFloat,A[i,j])
    end
    bb[i]=convert(BigFloat,b[i])
end
xb=Ab\bb

10-element Array{BigFloat,1}:
  7.086096455440146804509689118783702769154126827495550138924238137754096152196263e+07
  1.580422275885546389727987844856505092156359271397171187062791511633971663443229e+06
 -5.175489994935727685413969718989317356182318719998619656737629309183403633523838e+06
 -2.11069891625235027333227162014122074578125166016309593969821366165165842224424e+06 
  1.195910004115113399692296870223893932186573932558791159189569121120346698677215e+08
 -2.577283650921890931693621885131516669064466511633198125802767497613508030515626e+05
  3.659028196264402320190360219246187263387252138071620857302718088808222514044612e+06
 -1.771726043768045057458867151470849185671742068502712263097109162278267033062749e+08
 -1.098709142445369723038902222078316409640557179257769823886389690356418564986456e+07
  1.219781569728255440714468176388617449519739932160040971362976196947624778856822e+04

In [59]:
norm(xb-x)/norm(xb)

2.742728950333173979768028662629979375598101330704049401973633036524945694970124e-09

### Umjetno loša kondicija

In [60]:
A=[1 1; 1 2]
b=[1;3]
x=A\b
@show x,cond(A)
A1=[1e-4 1e-4;1 2]
b1=[1e-4;3]
x1=A1\b1
x,cond(A1),x-x1

(x,cond(A)) = ([-1.0,2.0],6.854101966249685)


([-1.0,2.0],50000.00017991671,[8.881784197001252e-16,-4.440892098500626e-16])

## Rezidual


Izračunato rješenje $\hat x$ sustava $Ax=b$ je točno rješenje nekog sličnog sustava (vidi [Afternotes on Numerical Analysis, str. 128][Ste96]):


$$ 
(A+\delta A)\hat x=b. \tag{1}
$$

__Rezidual__ (ili __ostatak__) definiramo kao 

$$
r=b-A\hat x.
$$

Tada je 

$$
0=b-(A+\delta A)\hat x=r- \delta A\hat x
$$

pa je 

$$ 
\| r\| \leq \| \delta A\hat x \| \leq \| \delta A\| \cdot \|\hat x \|,
$$

odnosno

$$
\frac{\|  \delta A\|}{\|A \|} \geq \frac{\|r\|}{\| A\| \cdot \|\hat x \|}.
$$

Dakle,

> ako  _relativni rezidual_ 
>
> $$ \frac{r}{\| A\| \cdot \|\hat x \|}$$
> 
> ima veliku normu, tada __rješenje nije izračunato stabilno.__

S druge strane, ako relativni rezidual ima malu normu, tada je rješenje izračunato stabilno. Naime, za

$$
\delta A=\frac{r\hat x^T}{\|\hat x\|^2}
$$

vrijedi (1):

$$
b-(A+\delta A)\hat x=(b-A\hat x)-\delta A \hat x = r-\frac{r\hat x^T \hat x}{\|\hat x\|^2}
= r-\frac{r \|\hat x^T \hat x\|}{\|\hat x\|^2}=r-r=0.
$$

Također vrijedi

$$
\frac{\|  \delta A\|}{\|A \|}  \leq  \frac{\|r\|\|\hat x \|}{\| A\| \cdot \|\hat x \|^2}=
\frac{\|r\|}{\| A\| \cdot \|\hat x \|}.
$$




[Ste96]: https://books.google.hr/books?id=w-2PWh01kWcC&printsec=frontcover&hl=hr#v=onepage&q&f=false    "G. W. Stewart, 'Afternotes on Numerical Analysis', SIAM, Philadelphia, 1996"

In [61]:
r=b-A*x

2-element Array{Float64,1}:
 0.0
 0.0

In [62]:
norm(r)/(norm(A)*norm(x))

0.0